# **Part 1: Define the NCR boundary and add it to the map**


In [ ]:
var ncr = ee.FeatureCollection('projects/ee-gauravdhyani/assets/Delhi-Shapefile'); // NCR boundary shapefile

// Add NCR boundary to the map

In [ ]:
Map.addLayer(ncr, {}, 'NCR'); // Display NCR boundary
Map.centerObject(ncr, 9); // Center the map on NCR

# **Part 2: Import the Sentinel-2 surface reflectance dataset**

In [ ]:
var dataset = ee.ImageCollection("COPERNICUS/S2_SR"); // Sentinel-2 surface reflectance

# **Part 3: Define a function to mask clouds and cirrus**

In [ ]:
function maskS2clouds(image) {
  var qa = image.select('QA60'); // Select the QA60 band
  var cloudBitMask = 1 << 10; // Bitmask for clouds
  var cirrusBitMask = 1 << 11; // Bitmask for cirrus
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0)
               .and(qa.bitwiseAnd(cirrusBitMask).eq(0)); // Create mask
  return image.updateMask(mask).divide(10000); // Apply mask and scale reflectance values
}

# **Part 4: Filter the Sentinel-2 data for 2023**

In [ ]:
var sentinel_2023 = dataset
  .filterBounds(ncr) // Filter by NCR boundary
  .filterDate('2023-01-01', '2023-12-01') // Filter by date range
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)) // Filter by cloud percentage
  .map(maskS2clouds) // Apply cloud mask
  .select('B.*') // Select all bands
  .median() // Calculate median
  .clip(ncr); // Clip to NCR boundary

# **Part 5: Filter the Sentinel-2 data for 2022**

In [ ]:
var sentinel_2022 = dataset
  .filterBounds(ncr) // Filter by NCR boundary
  .filterDate('2022-01-25', '2022-12-25') // Filter by date range
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)) // Filter by cloud percentage
  .map(maskS2clouds) // Apply cloud mask
  .select('B.*') // Select all bands
  .median() // Calculate median
  .clip(ncr); // Clip to NCR boundary

# **Part 6: Add filtered Sentinel-2 data to the map**

In [ ]:
Map.addLayer(sentinel_2023, {"opacity":1, "bands":["B4","B3","B2"], "min":0.1223, "max":0.2624, "gamma":1.8}, 'Sentinel 2023'); // Add 2023 layer
Map.addLayer(sentinel_2022, {"opacity":1, "bands":["B4","B3","B2"], "min":0.1241, "max":0.2562, "gamma":1.8}, 'Sentinel 2022'); // Add 2022 layer

# **Part 7: Merge training data from different classes**

In [ ]:
var training = Urban.merge(Water).merge(Vegetation).merge(Forest).merge(Bareland); // Combine training data

**Sample training data from 2023 image**

In [ ]:
var training = sentinel_2023.select('B.*')
  .sampleRegions({
    collection: training, // Training data
    properties: ['Class'], // Class property
    scale: 30, // Scale in meters
    tileScale: 16 // Tile scale for computation
  });

print(training); // Print training data

# **Part 8: Split data into 80% training and 20% testing**

In [ ]:
var split = 0.8; // Training data ratio
var withRandom = training.randomColumn('random'); // Add random column

**Filter training and testing data**

In [ ]:
var training_data = withRandom.filter(ee.Filter.lt('random', split)); // Training data
var testing_data = withRandom.filter(ee.Filter.gte('random', split)); // Testing data

# **Part 9: Train classifier using Random Forest**

In [ ]:
var classifier = ee.Classifier.smileRandomForest(50).train({
  features: training_data, // Training data
  classProperty: 'Class', // Class property
  inputProperties: sentinel_2023.bandNames() // Input properties (bands)
});


# **Part 10.1: Classify 2023 Sentinel-2 image**

In [ ]:
var sentinel_2023_classifier = sentinel_2023.classify(classifier); // Classify 2023 image
Map.addLayer(sentinel_2023_classifier, {min:0, max:4, palette:['red','blue','yellow','green','white']}, '2023_Classify'); // Add classified layer

# **Part 10.2: Classify 2022 Sentinel-2 image**

In [ ]:
var sentinel_2022_classifier = sentinel_2022.classify(classifier); // Classify 2022 image
Map.addLayer(sentinel_2022_classifier, {min:0, max:4, palette:['red','blue','yellow','green','white']}, '2022_Classify'); // Add classified layer

# **Part 11: Perform accuracy assessment**

In [ ]:
var testAccuracy = testing_data.classify(classifier).errorMatrix('Class', 'classification'); // Compute confusion matrix
print('Confusion Matrix:', testAccuracy); // Print confusion matrix
print('Overall Accuracy:', testAccuracy.accuracy()); // Print overall accuracy
print('Kappa Accuracy:', testAccuracy.kappa()); // Print kappa accuracy

# **Part 12: Perform change detection between 2022 and 2023**

In [ ]:
var landcover_2023 = sentinel_2023_classifier.remap([0,1,2,3,4], [1,2,3,4,5]); // Remap classes for 2023
var landcover_2022 = sentinel_2022_classifier.remap([0,1,2,3,4], [1,2,3,4,5]); // Remap classes for 2022
var changed = landcover_2022.subtract(landcover_2023).neq(0); // Detect changes
Map.addLayer(changed, {min:0, max:1, palette:['white','red']}, 'Change_2022_2023'); // Add change layer

# **Part 13: Calculate area of each class for 2022 and 2023**

In [ ]:
var pixelArea = ee.Image.pixelArea(); // Define pixel area in square meters
var classNames = ['Urban', 'Water', 'Vegetation', 'Forest', 'Bareland']; // Class names

**Function to calculate area for a given land cover class**

In [ ]:
function calculateArea(classifiedImage, className, region) {
  var area = classifiedImage.eq(classNames.indexOf(className) + 1) // Select class
    .multiply(pixelArea) // Multiply by pixel area
    .reduceRegion({
      reducer: ee.Reducer.sum(), // Sum areas
      geometry: region, // Region of interest
      scale: 10, // Scale in meters
      maxPixels: 1e12 // Maximum pixels
    });
  return ee.Number(area.get('remapped')); // Return area
}


**Calculate the area for each class in 2023**

In [ ]:
var area_2023 = {};
classNames.forEach(function(className) {
  area_2023[className] = calculateArea(landcover_2023, className, ncr); // Calculate area for 2023
});
print('Area 2023:', area_2023); // Print areas for 2023

**Calculate the area for each class in 2022**

In [ ]:
var area_2022 = {};
classNames.forEach(function(className) {
  area_2022[className] = calculateArea(landcover_2022, className, ncr); // Calculate area for 2022
});
print('Area 2022:', area_2022); // Print areas for 2022

# Feature Collection For **NCR**

In [ ]:
 var Urban =
    /* color: #d63000 */
    /* shown: false */
    ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([77.0708360410548, 28.546128209904126]),
            {
              "Class": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08132884272106, 28.543903990228035]),
            {
              "Class": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08265921839245, 28.541359274519234]),
            {
              "Class": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07684418924816, 28.54256566592624]),
            {
              "Class": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09632775553234, 28.542207519919224]),
            {
              "Class": 0,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09780833490856, 28.540341371023313]),
            {
              "Class": 0,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08199403055676, 28.553596083263436]),
            {
              "Class": 0,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07933327921398, 28.552615993953506]),
            {
              "Class": 0,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08506481510224, 28.55511863646544]),
            {
              "Class": 0,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07319872242036, 28.55709760652829]),
            {
              "Class": 0,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07234041553559, 28.557908031150912]),
            {
              "Class": 0,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07178714150241, 28.555507185885382]),
            {
              "Class": 0,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07026364678195, 28.557542689891118]),
            {
              "Class": 0,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07451226586154, 28.555111388869637]),
            {
              "Class": 0,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09616305702976, 28.554696742781065]),
            {
              "Class": 0,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08837392205051, 28.556166844271736]),
            {
              "Class": 0,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09288003319553, 28.561820889457767]),
            {
              "Class": 0,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10838972106161, 28.567148594694874]),
            {
              "Class": 0,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09373413100424, 28.572293251413928]),
            {
              "Class": 0,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11008487715903, 28.564510211627162]),
            {
              "Class": 0,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11770235076132, 28.56701667711247]),
            {
              "Class": 0,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11628614440146, 28.562437149957095]),
            {
              "Class": 0,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06006901724295, 28.60681011550913]),
            {
              "Class": 0,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05912487966971, 28.612386089465236]),
            {
              "Class": 0,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03108499990488, 28.618810824352753]),
            {
              "Class": 0,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0386113158709, 28.635737531572886]),
            {
              "Class": 0,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00462236323418, 28.62925875882408]),
            {
              "Class": 0,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04127206721367, 28.642291231893633]),
            {
              "Class": 0,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02831163325371, 28.63566219930439]),
            {
              "Class": 0,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0703949585911, 28.615025236319763]),
            {
              "Class": 0,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05151220712625, 28.622408993127287]),
            {
              "Class": 0,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05340048227274, 28.611483862626137]),
            {
              "Class": 0,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09571501169168, 28.60357185177196]),
            {
              "Class": 0,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08043714914285, 28.58066133716854]),
            {
              "Class": 0,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10833212289774, 28.587067740490006]),
            {
              "Class": 0,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04500270140156, 28.71070385323163]),
            {
              "Class": 0,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04671931517109, 28.69579767665135]),
            {
              "Class": 0,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06319880735859, 28.702121768474235]),
            {
              "Class": 0,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08396983396992, 28.709951066947145]),
            {
              "Class": 0,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05427241575703, 28.68103997648276]),
            {
              "Class": 0,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05238414061054, 28.697152871354643]),
            {
              "Class": 0,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03710627806171, 28.682696556663146]),
            {
              "Class": 0,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04551768553242, 28.675015827682444]),
            {
              "Class": 0,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04465937864765, 28.683148346710155]),
            {
              "Class": 0,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07332682859882, 28.67381095634931]),
            {
              "Class": 0,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06268382322773, 28.66387023916075]),
            {
              "Class": 0,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06663203489765, 28.647149633918975]),
            {
              "Class": 0,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06010890257343, 28.647149633918975]),
            {
              "Class": 0,
              "system:index": "47"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06869197142109, 28.67546765085984]),
            {
              "Class": 0,
              "system:index": "48"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06337046873554, 28.677274924082333]),
            {
              "Class": 0,
              "system:index": "49"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08482814085468, 28.692786070047465]),
            {
              "Class": 0,
              "system:index": "50"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08658916436937, 28.677909666685554]),
            {
              "Class": 0,
              "system:index": "51"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08517295800951, 28.683670107075137]),
            {
              "Class": 0,
              "system:index": "52"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08826286279466, 28.683406564430936]),
            {
              "Class": 0,
              "system:index": "53"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07229835473802, 28.66834588313741]),
            {
              "Class": 0,
              "system:index": "54"
            }),
        ee.Feature(
            ee.Geometry.Point([77.100751227968, 28.672261868563133]),
            {
              "Class": 0,
              "system:index": "55"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10345640882441, 28.6767718268803]),
            {
              "Class": 0,
              "system:index": "56"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10616635243572, 28.670782257881775]),
            {
              "Class": 0,
              "system:index": "57"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12882565419353, 28.672627264809247]),
            {
              "Class": 0,
              "system:index": "58"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13178681294598, 28.672439000284697]),
            {
              "Class": 0,
              "system:index": "59"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1192820707684, 28.680738496799478]),
            {
              "Class": 0,
              "system:index": "60"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1175648543427, 28.68135971922593]),
            {
              "Class": 0,
              "system:index": "61"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13008540602422, 28.686075242041206]),
            {
              "Class": 0,
              "system:index": "62"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12228056158001, 28.68666394995088]),
            {
              "Class": 0,
              "system:index": "63"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12117549146588, 28.68695101422637]),
            {
              "Class": 0,
              "system:index": "64"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1210278823141, 28.686895933499397]),
            {
              "Class": 0,
              "system:index": "65"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1326855025643, 28.685991364845748]),
            {
              "Class": 0,
              "system:index": "66"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15126784661948, 28.678386179118334]),
            {
              "Class": 0,
              "system:index": "67"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16148169854819, 28.676578925068075]),
            {
              "Class": 0,
              "system:index": "68"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16175716586882, 28.656602775135806]),
            {
              "Class": 0,
              "system:index": "69"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16716449924284, 28.661309998015042]),
            {
              "Class": 0,
              "system:index": "70"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16252964206511, 28.660632170947643]),
            {
              "Class": 0,
              "system:index": "71"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19541395227363, 28.66013210036445]),
            {
              "Class": 0,
              "system:index": "72"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19624007265021, 28.660273315371953]),
            {
              "Class": 0,
              "system:index": "73"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19727004091193, 28.66036745860461]),
            {
              "Class": 0,
              "system:index": "74"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19744170228888, 28.661421857033783]),
            {
              "Class": 0,
              "system:index": "75"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1962186149781, 28.66137478590151]),
            {
              "Class": 0,
              "system:index": "76"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19773138086249, 28.659444851278884]),
            {
              "Class": 0,
              "system:index": "77"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22638961305219, 28.660885244872823]),
            {
              "Class": 0,
              "system:index": "78"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20815059175092, 28.660489844680644]),
            {
              "Class": 0,
              "system:index": "79"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20806476106245, 28.65753371027365]),
            {
              "Class": 0,
              "system:index": "80"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20967408647138, 28.660828759222415]),
            {
              "Class": 0,
              "system:index": "81"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20606919755537, 28.65992498467602]),
            {
              "Class": 0,
              "system:index": "82"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21078988542158, 28.663031677033914]),
            {
              "Class": 0,
              "system:index": "83"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20806476106245, 28.662956364368803]),
            {
              "Class": 0,
              "system:index": "84"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2049349169281, 28.661934348336498]),
            {
              "Class": 0,
              "system:index": "85"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20750778588437, 28.663424337820043]),
            {
              "Class": 0,
              "system:index": "86"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20733390838937, 28.664494321405286]),
            {
              "Class": 0,
              "system:index": "87"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20820830852823, 28.664499028379378]),
            {
              "Class": 0,
              "system:index": "88"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21357826608366, 28.63289794718713]),
            {
              "Class": 0,
              "system:index": "89"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21418980973905, 28.632596609634838]),
            {
              "Class": 0,
              "system:index": "90"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21211584811253, 28.633865643915552]),
            {
              "Class": 0,
              "system:index": "91"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21146675353093, 28.633997477606925]),
            {
              "Class": 0,
              "system:index": "92"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21049042944951, 28.634181102829615]),
            {
              "Class": 0,
              "system:index": "93"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20877645600842, 28.626595508990498]),
            {
              "Class": 0,
              "system:index": "94"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20716444839047, 28.626028112092005]),
            {
              "Class": 0,
              "system:index": "95"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21243230689572, 28.62555253224605]),
            {
              "Class": 0,
              "system:index": "96"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21204847723602, 28.62484325730134]),
            {
              "Class": 0,
              "system:index": "97"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20416851644174, 28.62549541768057]),
            {
              "Class": 0,
              "system:index": "98"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20486589078561, 28.62578500384244]),
            {
              "Class": 0,
              "system:index": "99"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20568128232614, 28.626081652253117]),
            {
              "Class": 0,
              "system:index": "100"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20601119403497, 28.6260110217552]),
            {
              "Class": 0,
              "system:index": "101"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19299530166795, 28.62278107867909]),
            {
              "Class": 0,
              "system:index": "102"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19307995348917, 28.622128901438142]),
            {
              "Class": 0,
              "system:index": "103"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19277012879668, 28.62243135428384]),
            {
              "Class": 0,
              "system:index": "104"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19313372796672, 28.62252958157511]),
            {
              "Class": 0,
              "system:index": "105"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20407610335307, 28.623207787142793]),
            {
              "Class": 0,
              "system:index": "106"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2048002997871, 28.623377304784483]),
            {
              "Class": 0,
              "system:index": "107"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20511680045085, 28.623659833579048]),
            {
              "Class": 0,
              "system:index": "108"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21951006985003, 28.63296899294615]),
            {
              "Class": 0,
              "system:index": "109"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2212166628663, 28.637798391801457]),
            {
              "Class": 0,
              "system:index": "110"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2214339217965, 28.637374655410362]),
            {
              "Class": 0,
              "system:index": "111"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23143632601919, 28.625553893034997]),
            {
              "Class": 0,
              "system:index": "112"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25514897540285, 28.628303592808734]),
            {
              "Class": 0,
              "system:index": "113"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25456961825563, 28.628482519403335]),
            {
              "Class": 0,
              "system:index": "114"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25015470648275, 28.61676068291813]),
            {
              "Class": 0,
              "system:index": "115"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25079307222829, 28.616737137333935]),
            {
              "Class": 0,
              "system:index": "116"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25221282048985, 28.617869366522513]),
            {
              "Class": 0,
              "system:index": "117"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25261112852856, 28.61776812154185]),
            {
              "Class": 0,
              "system:index": "118"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25305905743404, 28.61768806730198]),
            {
              "Class": 0,
              "system:index": "119"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25202715333731, 28.615859667251023]),
            {
              "Class": 0,
              "system:index": "120"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25281572278769, 28.616059806220857]),
            {
              "Class": 0,
              "system:index": "121"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25324755843909, 28.616113961405933]),
            {
              "Class": 0,
              "system:index": "122"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24785991476034, 28.614623892617214]),
            {
              "Class": 0,
              "system:index": "123"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24639542863821, 28.614977082933155]),
            {
              "Class": 0,
              "system:index": "124"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24643297956442, 28.61413413341367]),
            {
              "Class": 0,
              "system:index": "125"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24141813059393, 28.615406947470834]),
            {
              "Class": 0,
              "system:index": "126"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23685073436802, 28.61257469547378]),
            {
              "Class": 0,
              "system:index": "127"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24002646984165, 28.612320392749258]),
            {
              "Class": 0,
              "system:index": "128"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2371350485236, 28.610069315859683]),
            {
              "Class": 0,
              "system:index": "129"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23603275015562, 28.60902210645757]),
            {
              "Class": 0,
              "system:index": "130"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2359764237663, 28.608739538280208]),
            {
              "Class": 0,
              "system:index": "131"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23234079020969, 28.61286311299714]),
            {
              "Class": 0,
              "system:index": "132"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23374090331546, 28.61275479924642]),
            {
              "Class": 0,
              "system:index": "133"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22417078155033, 28.613188053579076]),
            {
              "Class": 0,
              "system:index": "134"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21926735218099, 28.612135780979752]),
            {
              "Class": 0,
              "system:index": "135"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21745417888692, 28.615107322193182]),
            {
              "Class": 0,
              "system:index": "136"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2162793713384, 28.61631286803006]),
            {
              "Class": 0,
              "system:index": "137"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20991717155508, 28.61736770928504]),
            {
              "Class": 0,
              "system:index": "138"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20809863384298, 28.617174636454664]),
            {
              "Class": 0,
              "system:index": "139"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20676289375356, 28.618493175547428]),
            {
              "Class": 0,
              "system:index": "140"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20789478595785, 28.618893443067577]),
            {
              "Class": 0,
              "system:index": "141"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20697747047475, 28.619274873049388]),
            {
              "Class": 0,
              "system:index": "142"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20978842552236, 28.61683087175608]),
            {
              "Class": 0,
              "system:index": "143"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21034096057943, 28.617141672765246]),
            {
              "Class": 0,
              "system:index": "144"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20755146320394, 28.61668959826601]),
            {
              "Class": 0,
              "system:index": "145"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20584021385244, 28.615055521085427]),
            {
              "Class": 0,
              "system:index": "146"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20408068473867, 28.615389873240336]),
            {
              "Class": 0,
              "system:index": "147"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20743881042532, 28.614961337187648]),
            {
              "Class": 0,
              "system:index": "148"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21052335079244, 28.611909733199294]),
            {
              "Class": 0,
              "system:index": "149"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21101420418114, 28.61170072193877]),
            {
              "Class": 0,
              "system:index": "150"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20833199516625, 28.61109792427976]),
            {
              "Class": 0,
              "system:index": "151"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20780628219933, 28.610676434799082]),
            {
              "Class": 0,
              "system:index": "152"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20907496706337, 28.610167819542635]),
            {
              "Class": 0,
              "system:index": "153"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20957385794014, 28.611161500758552]),
            {
              "Class": 0,
              "system:index": "154"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20746564165444, 28.61070469112999]),
            {
              "Class": 0,
              "system:index": "155"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20952021375984, 28.611345165925545]),
            {
              "Class": 0,
              "system:index": "156"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2201874757883, 28.616049524539168]),
            {
              "Class": 0,
              "system:index": "157"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22141592751711, 28.616073070277558]),
            {
              "Class": 0,
              "system:index": "158"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22367434750765, 28.615955341532832]),
            {
              "Class": 0,
              "system:index": "159"
            })]),
    Water =
    /* color: #2b3ae7 */
    /* shown: false */
    ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([77.05997067302116, 28.69833716115584]),
            {
              "Class": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0606402175851, 28.695598346877343]),
            {
              "Class": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05657935313656, 28.707262821174012]),
            {
              "Class": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05391860179378, 28.70809090706786]),
            {
              "Class": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03705122078418, 28.71177759550398]),
            {
              "Class": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02913333977222, 28.71704689371306]),
            {
              "Class": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08860199751938, 28.811427421718864]),
            {
              "Class": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06950466933334, 28.814172395689955]),
            {
              "Class": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12163672327806, 28.8444746416595]),
            {
              "Class": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12638518532414, 28.851605492839308]),
            {
              "Class": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13075182160038, 28.84932197452134]),
            {
              "Class": 1,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1327070090809, 28.843394277492198]),
            {
              "Class": 1,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15190089679147, 28.828460048557634]),
            {
              "Class": 1,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13441289401437, 28.819474042619557]),
            {
              "Class": 1,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10235939131665, 28.7959750001315]),
            {
              "Class": 1,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13609085188794, 28.79966056357375]),
            {
              "Class": 1,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13493213759351, 28.798193875366604]),
            {
              "Class": 1,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0933257113545, 28.796539125431835]),
            {
              "Class": 1,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01372409623298, 28.817736896386634]),
            {
              "Class": 1,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9901567574125, 28.82283762374734]),
            {
              "Class": 1,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97353848215617, 28.815643786439942]),
            {
              "Class": 1,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00930842157878, 28.806656674553054]),
            {
              "Class": 1,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00995215174235, 28.8073523573413]),
            {
              "Class": 1,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96125590303275, 28.751938725544495]),
            {
              "Class": 1,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99975096681449, 28.73371515426189]),
            {
              "Class": 1,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00713240602347, 28.735295639938432]),
            {
              "Class": 1,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99386870616736, 28.74142261969387]),
            {
              "Class": 1,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02145574211403, 28.71376987570562]),
            {
              "Class": 1,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02422914623543, 28.71349229453024]),
            {
              "Class": 1,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99737524543654, 28.69304926709061]),
            {
              "Class": 1,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99843203578841, 28.69458331117425]),
            {
              "Class": 1,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0023909762944, 28.6904140468638]),
            {
              "Class": 1,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02888749601043, 28.694372356254956]),
            {
              "Class": 1,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96802297616328, 28.684180362237377]),
            {
              "Class": 1,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96983614945735, 28.681413144878505]),
            {
              "Class": 1,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97580620423193, 28.66753555387436]),
            {
              "Class": 1,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97981091955234, 28.668161279105203]),
            {
              "Class": 1,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Point([76.95268842199374, 28.65678901040929]),
            {
              "Class": 1,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Point([76.95091166426576, 28.661252730434015]),
            {
              "Class": 1,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01483873560719, 28.643222613275945]),
            {
              "Class": 1,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01076177790455, 28.641377088797093]),
            {
              "Class": 1,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Point([77.025610487011, 28.64092511866941]),
            {
              "Class": 1,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Point([77.009341946035, 28.625132207412353]),
            {
              "Class": 1,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98509477654036, 28.640688734282087]),
            {
              "Class": 1,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01414846458968, 28.62614929051226]),
            {
              "Class": 1,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02230237999495, 28.62377608128393]),
            {
              "Class": 1,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01172531950952, 28.612462865315333]),
            {
              "Class": 1,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0112532507229, 28.60372202411177]),
            {
              "Class": 1,
              "system:index": "47"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02215374815943, 28.58341183163806]),
            {
              "Class": 1,
              "system:index": "48"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03910530913355, 28.579699804690446]),
            {
              "Class": 1,
              "system:index": "49"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05946863997461, 28.59795719210646]),
            {
              "Class": 1,
              "system:index": "50"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05346418978732, 28.570413759450666]),
            {
              "Class": 1,
              "system:index": "51"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04185558917086, 28.56687088260847]),
            {
              "Class": 1,
              "system:index": "52"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06222160222094, 28.56878367401745]),
            {
              "Class": 1,
              "system:index": "53"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00224403696761, 28.564564794348403]),
            {
              "Class": 1,
              "system:index": "54"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00834332707791, 28.563470176194294]),
            {
              "Class": 1,
              "system:index": "55"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00776396993069, 28.56238652470646]),
            {
              "Class": 1,
              "system:index": "56"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00631600712035, 28.555587508658157]),
            {
              "Class": 1,
              "system:index": "57"
            }),
        ee.Feature(
            ee.Geometry.Point([76.90152198515486, 28.535605032094985]),
            {
              "Class": 1,
              "system:index": "58"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9279149218614, 28.53315437317905]),
            {
              "Class": 1,
              "system:index": "59"
            }),
        ee.Feature(
            ee.Geometry.Point([76.90195113859724, 28.521239588631055]),
            {
              "Class": 1,
              "system:index": "60"
            }),
        ee.Feature(
            ee.Geometry.Point([76.88377649031233, 28.53669838460919]),
            {
              "Class": 1,
              "system:index": "61"
            }),
        ee.Feature(
            ee.Geometry.Point([76.87894851408552, 28.538677700816766]),
            {
              "Class": 1,
              "system:index": "62"
            }),
        ee.Feature(
            ee.Geometry.Point([76.87478572569441, 28.536679533799873]),
            {
              "Class": 1,
              "system:index": "63"
            }),
        ee.Feature(
            ee.Geometry.Point([76.87403470717024, 28.53739586218228]),
            {
              "Class": 1,
              "system:index": "64"
            }),
        ee.Feature(
            ee.Geometry.Point([76.879935567003, 28.539884122932133]),
            {
              "Class": 1,
              "system:index": "65"
            }),
        ee.Feature(
            ee.Geometry.Point([76.8929389163072, 28.533192076055567]),
            {
              "Class": 1,
              "system:index": "66"
            }),
        ee.Feature(
            ee.Geometry.Point([76.89296037397932, 28.53413464358467]),
            {
              "Class": 1,
              "system:index": "67"
            }),
        ee.Feature(
            ee.Geometry.Point([76.90253049574446, 28.53027006313819]),
            {
              "Class": 1,
              "system:index": "68"
            }),
        ee.Feature(
            ee.Geometry.Point([76.90270215712141, 28.531156101420176]),
            {
              "Class": 1,
              "system:index": "69"
            }),
        ee.Feature(
            ee.Geometry.Point([76.91274183499202, 28.525730420035494]),
            {
              "Class": 1,
              "system:index": "70"
            }),
        ee.Feature(
            ee.Geometry.Point([76.91274183499202, 28.526842727570138]),
            {
              "Class": 1,
              "system:index": "71"
            }),
        ee.Feature(
            ee.Geometry.Point([76.93152652555726, 28.51034543766145]),
            {
              "Class": 1,
              "system:index": "72"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9249604778888, 28.51323029283477]),
            {
              "Class": 1,
              "system:index": "73"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96488439344844, 28.518213451575807]),
            {
              "Class": 1,
              "system:index": "74"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97352150988594, 28.5241900885708]),
            {
              "Class": 1,
              "system:index": "75"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9775126369001, 28.525472087837088]),
            {
              "Class": 1,
              "system:index": "76"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98420743060127, 28.525660615825966]),
            {
              "Class": 1,
              "system:index": "77"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98935727190987, 28.527093417522572]),
            {
              "Class": 1,
              "system:index": "78"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9948933513166, 28.527885220631045]),
            {
              "Class": 1,
              "system:index": "79"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00055817675606, 28.530863855237932]),
            {
              "Class": 1,
              "system:index": "80"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9986913592817, 28.53331456742091]),
            {
              "Class": 1,
              "system:index": "81"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0099137218, 28.544474012912456]),
            {
              "Class": 1,
              "system:index": "82"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00813273501412, 28.54253250514062]),
            {
              "Class": 1,
              "system:index": "83"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01227406573311, 28.545736916233395]),
            {
              "Class": 1,
              "system:index": "84"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06177508220841, 28.640431876412173]),
            {
              "Class": 1,
              "system:index": "85"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04327856884171, 28.65056312029943]),
            {
              "Class": 1,
              "system:index": "86"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04847132549455, 28.650638441867798]),
            {
              "Class": 1,
              "system:index": "87"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07168852672746, 28.64886837070884]),
            {
              "Class": 1,
              "system:index": "88"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08254610881974, 28.655722522654578]),
            {
              "Class": 1,
              "system:index": "89"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07211768016984, 28.654122005290038]),
            {
              "Class": 1,
              "system:index": "90"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09501301632096, 28.657360673949757]),
            {
              "Class": 1,
              "system:index": "91"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10651432857682, 28.66097581381566]),
            {
              "Class": 1,
              "system:index": "92"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11840187893083, 28.661804265809064]),
            {
              "Class": 1,
              "system:index": "93"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09552728468262, 28.671974744157705]),
            {
              "Class": 1,
              "system:index": "94"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11947630176238, 28.668538138218832]),
            {
              "Class": 1,
              "system:index": "95"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12150405177763, 28.669244154725444]),
            {
              "Class": 1,
              "system:index": "96"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11834977397612, 28.66927239528676]),
            {
              "Class": 1,
              "system:index": "97"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06573498835564, 28.675588123333114]),
            {
              "Class": 1,
              "system:index": "98"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0772363006115, 28.673611384818646]),
            {
              "Class": 1,
              "system:index": "99"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07807314982415, 28.665741713153768]),
            {
              "Class": 1,
              "system:index": "100"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07923393905881, 28.66149447701878]),
            {
              "Class": 1,
              "system:index": "101"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07613091007296, 28.685479261528773]),
            {
              "Class": 1,
              "system:index": "102"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07943539157931, 28.696508832483477]),
            {
              "Class": 1,
              "system:index": "103"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10175249012995, 28.71605098880402]),
            {
              "Class": 1,
              "system:index": "104"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11411313573721, 28.724360442127793]),
            {
              "Class": 1,
              "system:index": "105"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11456911126974, 28.72398409881192]),
            {
              "Class": 1,
              "system:index": "106"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11652767301268, 28.728209271067723]),
            {
              "Class": 1,
              "system:index": "107"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10345237376605, 28.73747472098814]),
            {
              "Class": 1,
              "system:index": "108"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11115567805682, 28.739073964287634]),
            {
              "Class": 1,
              "system:index": "109"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0987960589162, 28.725583109163466]),
            {
              "Class": 1,
              "system:index": "110"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09577020214877, 28.739976964435296]),
            {
              "Class": 1,
              "system:index": "111"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07608856694505, 28.750810848598388]),
            {
              "Class": 1,
              "system:index": "112"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17059534238983, 28.743170742853568]),
            {
              "Class": 1,
              "system:index": "113"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14307057321025, 28.78262127947282]),
            {
              "Class": 1,
              "system:index": "114"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11651026836529, 28.81594952805159]),
            {
              "Class": 1,
              "system:index": "115"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10343489555892, 28.814326950875213]),
            {
              "Class": 1,
              "system:index": "116"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11190473742731, 28.809313395575302]),
            {
              "Class": 1,
              "system:index": "117"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11266648478754, 28.807736380916452]),
            {
              "Class": 1,
              "system:index": "118"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12122223350175, 28.794754164695263]),
            {
              "Class": 1,
              "system:index": "119"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1540759787005, 28.46487335333341]),
            {
              "Class": 1,
              "system:index": "120"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19365613888868, 28.490180451151716]),
            {
              "Class": 1,
              "system:index": "121"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19743967501114, 28.497887173571467]),
            {
              "Class": 1,
              "system:index": "122"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13045508231177, 28.475840868754617]),
            {
              "Class": 1,
              "system:index": "123"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1400454576777, 28.49389789486539]),
            {
              "Class": 1,
              "system:index": "124"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17016859723545, 28.51495112146899]),
            {
              "Class": 1,
              "system:index": "125"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17761440946079, 28.513480444844113]),
            {
              "Class": 1,
              "system:index": "126"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17081232739902, 28.528563334046734]),
            {
              "Class": 1,
              "system:index": "127"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19199104978061, 28.554651635778097]),
            {
              "Class": 1,
              "system:index": "128"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10482306681571, 28.631368366652733]),
            {
              "Class": 1,
              "system:index": "129"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13185973368583, 28.628693926662972]),
            {
              "Class": 1,
              "system:index": "130"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13263220988212, 28.62255375733482]),
            {
              "Class": 1,
              "system:index": "131"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20838271121677, 28.62668138734854]),
            {
              "Class": 1,
              "system:index": "132"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22303830127414, 28.61522927398549]),
            {
              "Class": 1,
              "system:index": "133"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2311707590073, 28.612959422293947]),
            {
              "Class": 1,
              "system:index": "134"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21307700184646, 28.614514920846442]),
            {
              "Class": 1,
              "system:index": "135"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21321647671523, 28.61289493769896]),
            {
              "Class": 1,
              "system:index": "136"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2228831580049, 28.614015742656363]),
            {
              "Class": 1,
              "system:index": "137"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22073739079299, 28.612424007677827]),
            {
              "Class": 1,
              "system:index": "138"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22718542126479, 28.61380853591837]),
            {
              "Class": 1,
              "system:index": "139"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22586577442947, 28.61226389098964]),
            {
              "Class": 1,
              "system:index": "140"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2277483729696, 28.752873460869935]),
            {
              "Class": 1,
              "system:index": "141"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23564479630944, 28.729543978806767]),
            {
              "Class": 1,
              "system:index": "142"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18625246442579, 28.73194861028072]),
            {
              "Class": 1,
              "system:index": "143"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22045497892813, 28.714337476920097]),
            {
              "Class": 1,
              "system:index": "144"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22706394194083, 28.715767706249547]),
            {
              "Class": 1,
              "system:index": "145"
            }),
        ee.Feature(
            ee.Geometry.Point([77.21830921171622, 28.708277604111302]),
            {
              "Class": 1,
              "system:index": "146"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23504619596915, 28.712530843495948]),
            {
              "Class": 1,
              "system:index": "147"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22995926860669, 28.695852916439918]),
            {
              "Class": 1,
              "system:index": "148"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23251811644352, 28.681424623671766]),
            {
              "Class": 1,
              "system:index": "149"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25367538115299, 28.660225692835716]),
            {
              "Class": 1,
              "system:index": "150"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24911345100254, 28.648720730253398]),
            {
              "Class": 1,
              "system:index": "151"
            }),
        ee.Feature(
            ee.Geometry.Point([77.26200032700962, 28.653824218728914]),
            {
              "Class": 1,
              "system:index": "152"
            }),
        ee.Feature(
            ee.Geometry.Point([77.26450510688353, 28.64408511993515]),
            {
              "Class": 1,
              "system:index": "153"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25690909095336, 28.62913179419285]),
            {
              "Class": 1,
              "system:index": "154"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25551763352635, 28.61550887923114]),
            {
              "Class": 1,
              "system:index": "155"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25931564149144, 28.60262379314153]),
            {
              "Class": 1,
              "system:index": "156"
            }),
        ee.Feature(
            ee.Geometry.Point([77.26929345902684, 28.592676330143917]),
            {
              "Class": 1,
              "system:index": "157"
            }),
        ee.Feature(
            ee.Geometry.Point([77.28066602524999, 28.587381936637833]),
            {
              "Class": 1,
              "system:index": "158"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2725529979499, 28.584828188712116]),
            {
              "Class": 1,
              "system:index": "159"
            }),
        ee.Feature(
            ee.Geometry.Point([77.27548197019416, 28.584884715267084]),
            {
              "Class": 1,
              "system:index": "160"
            }),
        ee.Feature(
            ee.Geometry.Point([77.30670873000118, 28.54054965085819]),
            {
              "Class": 1,
              "system:index": "161"
            }),
        ee.Feature(
            ee.Geometry.Point([77.30915490462276, 28.543311157028555]),
            {
              "Class": 1,
              "system:index": "162"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15167944821768, 28.668004439126253]),
            {
              "Class": 1,
              "system:index": "163"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17148487958364, 28.67722939343592]),
            {
              "Class": 1,
              "system:index": "164"
            })]),
    Vegetation =
    /* color: #dbe315 */
    /* shown: false */
    ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([76.93824098887465, 28.614958468866444]),
            {
              "Class": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([76.94296167674086, 28.62023263720594]),
            {
              "Class": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([76.94721029582045, 28.627465351574905]),
            {
              "Class": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9459228354933, 28.63884080015743]),
            {
              "Class": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99141310038588, 28.674992974235852]),
            {
              "Class": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9889704158693, 28.664133057800402]),
            {
              "Class": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00441993979508, 28.676634078654452]),
            {
              "Class": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00193084982926, 28.654238496974813]),
            {
              "Class": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01304592398698, 28.648080991587324]),
            {
              "Class": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04353797097933, 28.665880979316697]),
            {
              "Class": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02609288354647, 28.66851681683937]),
            {
              "Class": 2,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0261357988907, 28.665824496358535]),
            {
              "Class": 2,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96094352279935, 28.706068194602004]),
            {
              "Class": 2,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([76.95571922496285, 28.709022433524176]),
            {
              "Class": 2,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([76.95215725139107, 28.71278634418672]),
            {
              "Class": 2,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96936630443062, 28.717980318481686]),
            {
              "Class": 2,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97854461148216, 28.72446634020413]),
            {
              "Class": 2,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97923125698998, 28.71791777985677]),
            {
              "Class": 2,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97793323558942, 28.719510247909597]),
            {
              "Class": 2,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98604423565045, 28.725635329074777]),
            {
              "Class": 2,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98874408840857, 28.72865938151976]),
            {
              "Class": 2,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99005300640783, 28.727577435328428]),
            {
              "Class": 2,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99222023129187, 28.727191695282695]),
            {
              "Class": 2,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98970968365393, 28.72422803525899]),
            {
              "Class": 2,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98858882173357, 28.720857884165884]),
            {
              "Class": 2,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98926473840532, 28.721413006680294]),
            {
              "Class": 2,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99711824640093, 28.722193936753595]),
            {
              "Class": 2,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00138101290644, 28.723373023187946]),
            {
              "Class": 2,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9920503658633, 28.722734107250837]),
            {
              "Class": 2,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99133113131958, 28.71528340039603]),
            {
              "Class": 2,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99349835620362, 28.71540572229984]),
            {
              "Class": 2,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97717535418766, 28.742530154852627]),
            {
              "Class": 2,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9794498674323, 28.741533015478286]),
            {
              "Class": 2,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98245394152897, 28.742473713255368]),
            {
              "Class": 2,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98532926959294, 28.74693488755043]),
            {
              "Class": 2,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99110138339299, 28.744338651989025]),
            {
              "Class": 2,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98818313998478, 28.743002882465944]),
            {
              "Class": 2,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9864021531989, 28.745411017977883]),
            {
              "Class": 2,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99384796542424, 28.74439509257869]),
            {
              "Class": 2,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99166864482137, 28.750817145769012]),
            {
              "Class": 2,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99323505488607, 28.750723083886317]),
            {
              "Class": 2,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99057365493361, 28.755262920794788]),
            {
              "Class": 2,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99183965758864, 28.75733217267692]),
            {
              "Class": 2,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98752989854975, 28.757165706103777]),
            {
              "Class": 2,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98883881654902, 28.757222139761513]),
            {
              "Class": 2,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98570555952263, 28.750561736158566]),
            {
              "Class": 2,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98740971159353, 28.76736273029415]),
            {
              "Class": 2,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99202311109914, 28.768096293800628]),
            {
              "Class": 2,
              "system:index": "47"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98204529356374, 28.772159013700815]),
            {
              "Class": 2,
              "system:index": "48"
            }),
        ee.Feature(
            ee.Geometry.Point([76.981228706216, 28.77724662423125]),
            {
              "Class": 2,
              "system:index": "49"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97770964798846, 28.772901976987175]),
            {
              "Class": 2,
              "system:index": "50"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9806708067409, 28.77624981639856]),
            {
              "Class": 2,
              "system:index": "51"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98869597611346, 28.773372185636223]),
            {
              "Class": 2,
              "system:index": "52"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97650701051738, 28.780902286408182]),
            {
              "Class": 2,
              "system:index": "53"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9918023685631, 28.77975040515116]),
            {
              "Class": 2,
              "system:index": "54"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99484026662137, 28.78088878373271]),
            {
              "Class": 2,
              "system:index": "55"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98170524223909, 28.785080081800203]),
            {
              "Class": 2,
              "system:index": "56"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98975186928377, 28.783857672114813]),
            {
              "Class": 2,
              "system:index": "57"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98760610207185, 28.787073519166007]),
            {
              "Class": 2,
              "system:index": "58"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98595386131868, 28.78350251697588]),
            {
              "Class": 2,
              "system:index": "59"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98591094597444, 28.784894186996926]),
            {
              "Class": 2,
              "system:index": "60"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01429469131105, 28.774621293004422]),
            {
              "Class": 2,
              "system:index": "61"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01813561462038, 28.776088312805335]),
            {
              "Class": 2,
              "system:index": "62"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01526028655641, 28.770840800755437]),
            {
              "Class": 2,
              "system:index": "63"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00107676528566, 28.776539699363084]),
            {
              "Class": 2,
              "system:index": "64"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00154883407228, 28.774621293004422]),
            {
              "Class": 2,
              "system:index": "65"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00356585525148, 28.777160352690952]),
            {
              "Class": 2,
              "system:index": "66"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00491768859499, 28.77300378572355]),
            {
              "Class": 2,
              "system:index": "67"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02262026809328, 28.772232639704264]),
            {
              "Class": 2,
              "system:index": "68"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02191216491335, 28.774903413798985]),
            {
              "Class": 2,
              "system:index": "69"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01907975219362, 28.7701824920828]),
            {
              "Class": 2,
              "system:index": "70"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02231986068361, 28.770897227019827]),
            {
              "Class": 2,
              "system:index": "71"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02207178209402, 28.766366525945706]),
            {
              "Class": 2,
              "system:index": "72"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02790826891042, 28.770617401360216]),
            {
              "Class": 2,
              "system:index": "73"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03771442506887, 28.774642406959707]),
            {
              "Class": 2,
              "system:index": "74"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03654506501408, 28.77096182784213]),
            {
              "Class": 2,
              "system:index": "75"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03549363908024, 28.77084897535265]),
            {
              "Class": 2,
              "system:index": "76"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03607299622746, 28.768949273460663]),
            {
              "Class": 2,
              "system:index": "77"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03223207291813, 28.76667334744259]),
            {
              "Class": 2,
              "system:index": "78"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03721025284977, 28.770943019102344]),
            {
              "Class": 2,
              "system:index": "79"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03991391953679, 28.778560281328275]),
            {
              "Class": 2,
              "system:index": "80"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03000047501774, 28.778447437057707]),
            {
              "Class": 2,
              "system:index": "81"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0388596366757, 28.774253305088767]),
            {
              "Class": 2,
              "system:index": "82"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04567542592125, 28.780723106254978]),
            {
              "Class": 2,
              "system:index": "83"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04092196084142, 28.78147538267378]),
            {
              "Class": 2,
              "system:index": "84"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04244545556188, 28.781851518848498]),
            {
              "Class": 2,
              "system:index": "85"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03957012749791, 28.781907939157723]),
            {
              "Class": 2,
              "system:index": "86"
            }),
        ee.Feature(
            ee.Geometry.Point([77.04763821221471, 28.779745138799125]),
            {
              "Class": 2,
              "system:index": "87"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06263712502599, 28.778503859208246]),
            {
              "Class": 2,
              "system:index": "88"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06456831551671, 28.77812771096149]),
            {
              "Class": 2,
              "system:index": "89"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06425717927098, 28.776829989098488]),
            {
              "Class": 2,
              "system:index": "90"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06516913033605, 28.77662310441099]),
            {
              "Class": 2,
              "system:index": "91"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06397822953343, 28.77812771096149]),
            {
              "Class": 2,
              "system:index": "92"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06607035256505, 28.777046277196753]),
            {
              "Class": 2,
              "system:index": "93"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06362958205682, 28.789022046785337]),
            {
              "Class": 2,
              "system:index": "94"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0635008360241, 28.78766804331064]),
            {
              "Class": 2,
              "system:index": "95"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06566806090814, 28.78482834003249]),
            {
              "Class": 2,
              "system:index": "96"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06680531753045, 28.7873671512619]),
            {
              "Class": 2,
              "system:index": "97"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07026000274163, 28.78559939294124]),
            {
              "Class": 2,
              "system:index": "98"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06816787971002, 28.782693810172063]),
            {
              "Class": 2,
              "system:index": "99"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06892962707025, 28.78288187623193]),
            {
              "Class": 2,
              "system:index": "100"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0675509205716, 28.792878561911618]),
            {
              "Class": 2,
              "system:index": "101"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0703404179471, 28.790875832953525]),
            {
              "Class": 2,
              "system:index": "102"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07168152245454, 28.79156221857279]),
            {
              "Class": 2,
              "system:index": "103"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07005073937349, 28.788421732341426]),
            {
              "Class": 2,
              "system:index": "104"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0691709748166, 28.789098731383064]),
            {
              "Class": 2,
              "system:index": "105"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07287242325715, 28.78866620472463]),
            {
              "Class": 2,
              "system:index": "106"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07390239151887, 28.79112970213381]),
            {
              "Class": 2,
              "system:index": "107"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0822816124814, 28.78912693958106]),
            {
              "Class": 2,
              "system:index": "108"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08668471346553, 28.790079824874695]),
            {
              "Class": 2,
              "system:index": "109"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09105134974178, 28.790117435449787]),
            {
              "Class": 2,
              "system:index": "110"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11511111683426, 28.8080848615545]),
            {
              "Class": 2,
              "system:index": "111"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11712813801346, 28.809701824666462]),
            {
              "Class": 2,
              "system:index": "112"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1153900665718, 28.81009666068381]),
            {
              "Class": 2,
              "system:index": "113"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10833049244461, 28.81092393129636]),
            {
              "Class": 2,
              "system:index": "114"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10710740513382, 28.810453891981496]),
            {
              "Class": 2,
              "system:index": "115"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10467732376632, 28.805527752387576]),
            {
              "Class": 2,
              "system:index": "116"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1050689262825, 28.80579568772526]),
            {
              "Class": 2,
              "system:index": "117"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10488117165146, 28.80488846529012]),
            {
              "Class": 2,
              "system:index": "118"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10529423183975, 28.80501538289357]),
            {
              "Class": 2,
              "system:index": "119"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10588968224106, 28.804883764635154]),
            {
              "Class": 2,
              "system:index": "120"
            }),
        ee.Feature(
            ee.Geometry.Point([77.105160121389, 28.80447480684288]),
            {
              "Class": 2,
              "system:index": "121"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10742390579757, 28.805123497767156]),
            {
              "Class": 2,
              "system:index": "122"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10955894417343, 28.805617064243375]),
            {
              "Class": 2,
              "system:index": "123"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10947847790298, 28.806110628381603]),
            {
              "Class": 2,
              "system:index": "124"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11063719219742, 28.806237544496405]),
            {
              "Class": 2,
              "system:index": "125"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12177372402725, 28.803633382888588]),
            {
              "Class": 2,
              "system:index": "126"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12076521343765, 28.803407748494184]),
            {
              "Class": 2,
              "system:index": "127"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11907005734024, 28.800606080736415]),
            {
              "Class": 2,
              "system:index": "128"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11900568432388, 28.802524009078724]),
            {
              "Class": 2,
              "system:index": "129"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12181663937149, 28.801696671769896]),
            {
              "Class": 2,
              "system:index": "130"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13095760769424, 28.793799030531368]),
            {
              "Class": 2,
              "system:index": "131"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12885475582657, 28.790526975288603]),
            {
              "Class": 2,
              "system:index": "132"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12885475582657, 28.793065647768355]),
            {
              "Class": 2,
              "system:index": "133"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13209486431656, 28.790583390904196]),
            {
              "Class": 2,
              "system:index": "134"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13125801510391, 28.79285879528703]),
            {
              "Class": 2,
              "system:index": "135"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14220142788467, 28.78755570972805]),
            {
              "Class": 2,
              "system:index": "136"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14198685116348, 28.785167352613026]),
            {
              "Class": 2,
              "system:index": "137"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14243746227798, 28.783380750504495]),
            {
              "Class": 2,
              "system:index": "138"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14990473217544, 28.7848288408795]),
            {
              "Class": 2,
              "system:index": "139"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14456177181778, 28.787160788310562]),
            {
              "Class": 2,
              "system:index": "140"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14428282208023, 28.785656312065964]),
            {
              "Class": 2,
              "system:index": "141"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14374638027725, 28.784321071218987]),
            {
              "Class": 2,
              "system:index": "142"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14754438824234, 28.786634224093984]),
            {
              "Class": 2,
              "system:index": "143"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15084886974869, 28.787630932685605]),
            {
              "Class": 2,
              "system:index": "144"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14546299404678, 28.788590020637738]),
            {
              "Class": 2,
              "system:index": "145"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16224289364395, 28.785487057130148]),
            {
              "Class": 2,
              "system:index": "146"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1873054546791, 28.789097769511574]),
            {
              "Class": 2,
              "system:index": "147"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1754179043251, 28.79473925729555]),
            {
              "Class": 2,
              "system:index": "148"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17565248203124, 28.793953051525236]),
            {
              "Class": 2,
              "system:index": "149"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17198322009887, 28.799500258650795]),
            {
              "Class": 2,
              "system:index": "150"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19863364887084, 28.799537865827176]),
            {
              "Class": 2,
              "system:index": "151"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19571540546264, 28.799312222565387]),
            {
              "Class": 2,
              "system:index": "152"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19704578113402, 28.797356627166728]),
            {
              "Class": 2,
              "system:index": "153"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20650861453856, 28.796980546922008]),
            {
              "Class": 2,
              "system:index": "154"
            })]),
    Forest =
    /* color: #30742a */
    /* shown: false */
    ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([77.11723736489643, 28.85570949279068]),
            {
              "Class": 3,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12122849191059, 28.85409324426809]),
            {
              "Class": 3,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12133578027118, 28.849639034033803]),
            {
              "Class": 3,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12260412452336, 28.85613414911826]),
            {
              "Class": 3,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11185383079167, 28.85660398217334]),
            {
              "Class": 3,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11339878318425, 28.856434842518077]),
            {
              "Class": 3,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11208986518498, 28.855833454848884]),
            {
              "Class": 3,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11990045783635, 28.850138892970573]),
            {
              "Class": 3,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1221964287531, 28.85177399710636]),
            {
              "Class": 3,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12869091667432, 28.792615218294443]),
            {
              "Class": 3,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12887330688733, 28.790668899846843]),
            {
              "Class": 3,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13205977119702, 28.790480847830388]),
            {
              "Class": 3,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13195248283643, 28.79460851160211]),
            {
              "Class": 3,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12987108864087, 28.794702533930053]),
            {
              "Class": 3,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13202758468884, 28.792737449641322]),
            {
              "Class": 3,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2351068236005, 28.703200409697992]),
            {
              "Class": 3,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23407685533878, 28.701130085909185]),
            {
              "Class": 3,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23405539766667, 28.697328111918647]),
            {
              "Class": 3,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23339020983097, 28.69354482249235]),
            {
              "Class": 3,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23334729448673, 28.688979713323636]),
            {
              "Class": 3,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2281545378339, 28.692612585098725]),
            {
              "Class": 3,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22918450609562, 28.692687876444573]),
            {
              "Class": 3,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22948491350529, 28.68306896862914]),
            {
              "Class": 3,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23023593202946, 28.683125442290795]),
            {
              "Class": 3,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22914159075138, 28.68711617059624]),
            {
              "Class": 3,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Point([77.22976386324284, 28.68739852768691]),
            {
              "Class": 3,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2281545378339, 28.690222056715225]),
            {
              "Class": 3,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2292703367841, 28.690485582206165]),
            {
              "Class": 3,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23789901850348, 28.703021099869247]),
            {
              "Class": 3,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24040956614142, 28.70266350080125]),
            {
              "Class": 3,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2435209285987, 28.70379275685215]),
            {
              "Class": 3,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23752426851335, 28.701148618451054]),
            {
              "Class": 3,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24160122621599, 28.700527513437216]),
            {
              "Class": 3,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23816799867693, 28.695803230413567]),
            {
              "Class": 3,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23559307802263, 28.691323434413146]),
            {
              "Class": 3,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24398302782122, 28.69727135713888]),
            {
              "Class": 3,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24140897927722, 28.69211132832805]),
            {
              "Class": 3,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Point([77.244970952849, 28.691772514979338]),
            {
              "Class": 3,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24219940357372, 28.684621535535257]),
            {
              "Class": 3,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24908731632397, 28.691398057006968]),
            {
              "Class": 3,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24003217868969, 28.68625923507812]),
            {
              "Class": 3,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Point([77.24648957376255, 28.702444217839133]),
            {
              "Class": 3,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14738701676971, 28.61306426528152]),
            {
              "Class": 3,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14748141282017, 28.606690350711897]),
            {
              "Class": 3,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15082880967076, 28.60714246824981]),
            {
              "Class": 3,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17224356644566, 28.62375643795479]),
            {
              "Class": 3,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15730902665074, 28.610903405586782]),
            {
              "Class": 3,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15567824356968, 28.617760037517392]),
            {
              "Class": 3,
              "system:index": "47"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16065642350132, 28.618814864235677]),
            {
              "Class": 3,
              "system:index": "48"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16340300553257, 28.612485744993744]),
            {
              "Class": 3,
              "system:index": "49"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1567082118314, 28.607738655262978]),
            {
              "Class": 3,
              "system:index": "50"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18906638138706, 28.624088838881256]),
            {
              "Class": 3,
              "system:index": "51"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18383070939, 28.627780463499946]),
            {
              "Class": 3,
              "system:index": "52"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18091246598179, 28.611204805406853]),
            {
              "Class": 3,
              "system:index": "53"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16666457169468, 28.611204805406853]),
            {
              "Class": 3,
              "system:index": "54"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1827149104398, 28.605704122565765]),
            {
              "Class": 3,
              "system:index": "55"
            }),
        ee.Feature(
            ee.Geometry.Point([77.172844381265, 28.60133351169978]),
            {
              "Class": 3,
              "system:index": "56"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18185660355503, 28.619945023961193]),
            {
              "Class": 3,
              "system:index": "57"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1900105189603, 28.627554449397028]),
            {
              "Class": 3,
              "system:index": "58"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16743704789097, 28.625746319054592]),
            {
              "Class": 3,
              "system:index": "59"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16117140763218, 28.608718231027776]),
            {
              "Class": 3,
              "system:index": "60"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16014143937046, 28.617918371810674]),
            {
              "Class": 3,
              "system:index": "61"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16082808487828, 28.613472920927254]),
            {
              "Class": 3,
              "system:index": "62"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18889472001011, 28.61972663697094]),
            {
              "Class": 3,
              "system:index": "63"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18202826493199, 28.616034729156706]),
            {
              "Class": 3,
              "system:index": "64"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18353460462797, 28.62278179064789]),
            {
              "Class": 3,
              "system:index": "65"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18430708082425, 28.63174705113832]),
            {
              "Class": 3,
              "system:index": "66"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17658231886136, 28.62782955251244]),
            {
              "Class": 3,
              "system:index": "67"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17529485853422, 28.62451463178125]),
            {
              "Class": 3,
              "system:index": "68"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16885755689847, 28.607561696508043]),
            {
              "Class": 3,
              "system:index": "69"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1702308479141, 28.613740416522955]),
            {
              "Class": 3,
              "system:index": "70"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1728057685684, 28.606506756769303]),
            {
              "Class": 3,
              "system:index": "71"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19801147847048, 28.641631209400785]),
            {
              "Class": 3,
              "system:index": "72"
            }),
        ee.Feature(
            ee.Geometry.Point([77.2005005684363, 28.641085079907658]),
            {
              "Class": 3,
              "system:index": "73"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1958013382422, 28.63997398009405]),
            {
              "Class": 3,
              "system:index": "74"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19181021122805, 28.639258350255112]),
            {
              "Class": 3,
              "system:index": "75"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19400549530309, 28.64234972469776]),
            {
              "Class": 3,
              "system:index": "76"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19669361288382, 28.636440262028056]),
            {
              "Class": 3,
              "system:index": "77"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19971914465262, 28.638982677346746]),
            {
              "Class": 3,
              "system:index": "78"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1948697107537, 28.63470761777406]),
            {
              "Class": 3,
              "system:index": "79"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19517011816336, 28.636233099547002]),
            {
              "Class": 3,
              "system:index": "80"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18990237541995, 28.636607781936156]),
            {
              "Class": 3,
              "system:index": "81"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19134003945193, 28.6387170460016]),
            {
              "Class": 3,
              "system:index": "82"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19389350243411, 28.63331197194946]),
            {
              "Class": 3,
              "system:index": "83"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19732672997317, 28.634140642487424]),
            {
              "Class": 3,
              "system:index": "84"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18657643624148, 28.63261513029586]),
            {
              "Class": 3,
              "system:index": "85"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17837260599883, 28.629796973529295]),
            {
              "Class": 3,
              "system:index": "86"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18193457957061, 28.62548386995618]),
            {
              "Class": 3,
              "system:index": "87"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18238519068511, 28.629175445515912]),
            {
              "Class": 3,
              "system:index": "88"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18483136530669, 28.629796973529295]),
            {
              "Class": 3,
              "system:index": "89"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18689130183013, 28.62245140719985]),
            {
              "Class": 3,
              "system:index": "90"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1886508309439, 28.626971816539868]),
            {
              "Class": 3,
              "system:index": "91"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19299961783996, 28.629426574008516]),
            {
              "Class": 3,
              "system:index": "92"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18926598289123, 28.622250501220652]),
            {
              "Class": 3,
              "system:index": "93"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18480278709045, 28.61833264818862]),
            {
              "Class": 3,
              "system:index": "94"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18892037006916, 28.616101853455394]),
            {
              "Class": 3,
              "system:index": "95"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18962605485149, 28.61362176411808]),
            {
              "Class": 3,
              "system:index": "96"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18945439347453, 28.61006150543776]),
            {
              "Class": 3,
              "system:index": "97"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18771632203288, 28.607838637241407]),
            {
              "Class": 3,
              "system:index": "98"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1864528251004, 28.606745611142035]),
            {
              "Class": 3,
              "system:index": "99"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18066289718224, 28.604591833316434]),
            {
              "Class": 3,
              "system:index": "100"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17672005950062, 28.60944730690348]),
            {
              "Class": 3,
              "system:index": "101"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1695993028574, 28.61118877080271]),
            {
              "Class": 3,
              "system:index": "102"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1638701044016, 28.61403318944317]),
            {
              "Class": 3,
              "system:index": "103"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1504161439829, 28.610246893349732]),
            {
              "Class": 3,
              "system:index": "104"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1475890956812, 28.620102438014335]),
            {
              "Class": 3,
              "system:index": "105"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14212109397639, 28.61841849654021]),
            {
              "Class": 3,
              "system:index": "106"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14368750404108, 28.619322628706595]),
            {
              "Class": 3,
              "system:index": "107"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14429944804219, 28.626731559411734]),
            {
              "Class": 3,
              "system:index": "108"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14523285677937, 28.62417001215476]),
            {
              "Class": 3,
              "system:index": "109"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14675635149983, 28.623247086429352]),
            {
              "Class": 3,
              "system:index": "110"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14928835680989, 28.623105821571738]),
            {
              "Class": 3,
              "system:index": "111"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14950293353108, 28.620949154477366]),
            {
              "Class": 3,
              "system:index": "112"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14781850626973, 28.622653772750038]),
            {
              "Class": 3,
              "system:index": "113"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13309826462084, 28.541317348524135]),
            {
              "Class": 3,
              "system:index": "114"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13505091278368, 28.545784699331076]),
            {
              "Class": 3,
              "system:index": "115"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1385485133391, 28.543371976434564]),
            {
              "Class": 3,
              "system:index": "116"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1420246562224, 28.544785687960996]),
            {
              "Class": 3,
              "system:index": "117"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13908495514208, 28.540864948004614]),
            {
              "Class": 3,
              "system:index": "118"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1440416774016, 28.542806486511314]),
            {
              "Class": 3,
              "system:index": "119"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1350723704558, 28.557828637708916]),
            {
              "Class": 3,
              "system:index": "120"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13374199478442, 28.557376308115177]),
            {
              "Class": 3,
              "system:index": "121"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13599505035693, 28.55469998491818]),
            {
              "Class": 3,
              "system:index": "122"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14035095779711, 28.55409686043827]),
            {
              "Class": 3,
              "system:index": "123"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14105906097704, 28.556433948552694]),
            {
              "Class": 3,
              "system:index": "124"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14889395205196, 28.559296200369086]),
            {
              "Class": 3,
              "system:index": "125"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15181219546017, 28.5578826836412]),
            {
              "Class": 3,
              "system:index": "126"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1534215208691, 28.55302004115494]),
            {
              "Class": 3,
              "system:index": "127"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15305674044308, 28.55430168965279]),
            {
              "Class": 3,
              "system:index": "128"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1471344229382, 28.552039946483127]),
            {
              "Class": 3,
              "system:index": "129"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1449028250378, 28.552944649580983]),
            {
              "Class": 3,
              "system:index": "130"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15157616106686, 28.555149830815775]),
            {
              "Class": 3,
              "system:index": "131"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15305674044308, 28.549325790551148]),
            {
              "Class": 3,
              "system:index": "132"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15561020342525, 28.548684937969767]),
            {
              "Class": 3,
              "system:index": "133"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14870083300289, 28.540108447269958]),
            {
              "Class": 3,
              "system:index": "134"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15124903516221, 28.537791093885495]),
            {
              "Class": 3,
              "system:index": "135"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14650688962388, 28.547065186813754]),
            {
              "Class": 3,
              "system:index": "136"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15116320447373, 28.544972932296094]),
            {
              "Class": 3,
              "system:index": "137"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16139851407456, 28.55160768641693]),
            {
              "Class": 3,
              "system:index": "138"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16766415433335, 28.55102339246979]),
            {
              "Class": 3,
              "system:index": "139"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17150507764268, 28.549854794849253]),
            {
              "Class": 3,
              "system:index": "140"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17440186337876, 28.548761578883415]),
            {
              "Class": 3,
              "system:index": "141"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17654763059068, 28.54518027468917]),
            {
              "Class": 3,
              "system:index": "142"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17778671492361, 28.540727104050607]),
            {
              "Class": 3,
              "system:index": "143"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16840971220755, 28.54297024049606]),
            {
              "Class": 3,
              "system:index": "144"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18242157210135, 28.536297240524302]),
            {
              "Class": 3,
              "system:index": "145"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18675602186941, 28.535354692339585]),
            {
              "Class": 3,
              "system:index": "146"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17154253233694, 28.526984494624966]),
            {
              "Class": 3,
              "system:index": "147"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17544782866263, 28.527191872418765]),
            {
              "Class": 3,
              "system:index": "148"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18018997420096, 28.52713531487909]),
            {
              "Class": 3,
              "system:index": "149"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18109119642996, 28.524552488232107]),
            {
              "Class": 3,
              "system:index": "150"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18476045836233, 28.529190219340656]),
            {
              "Class": 3,
              "system:index": "151"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18744266737723, 28.531697525564244]),
            {
              "Class": 3,
              "system:index": "152"
            }),
        ee.Feature(
            ee.Geometry.Point([77.18804348219656, 28.533790043738534]),
            {
              "Class": 3,
              "system:index": "153"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15491283644461, 28.534196779030086]),
            {
              "Class": 3,
              "system:index": "154"
            }),
        ee.Feature(
            ee.Geometry.Point([77.15892542113089, 28.532085417301072]),
            {
              "Class": 3,
              "system:index": "155"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16338861693167, 28.528861738779653]),
            {
              "Class": 3,
              "system:index": "156"
            }),
        ee.Feature(
            ee.Geometry.Point([77.17834461439871, 28.53398941501931]),
            {
              "Class": 3,
              "system:index": "157"
            }),
        ee.Feature(
            ee.Geometry.Point([77.16401088942312, 28.535271295095512]),
            {
              "Class": 3,
              "system:index": "158"
            }),
        ee.Feature(
            ee.Geometry.Point([77.14881885756277, 28.55012488550119]),
            {
              "Class": 3,
              "system:index": "159"
            })]),
    Bareland =
    /* color: #f5fdff */
    /* shown: false */
    ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([77.0101690539936, 28.671203244764367]),
            {
              "Class": 4,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([76.94420595037751, 28.660782098555735]),
            {
              "Class": 4,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([76.94514472353272, 28.66098450541899]),
            {
              "Class": 4,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([76.946766290793, 28.66008358291628]),
            {
              "Class": 4,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9577696203365, 28.662906002536136]),
            {
              "Class": 4,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([76.95900343648336, 28.662461185790534]),
            {
              "Class": 4,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96174898223722, 28.661425608420885]),
            {
              "Class": 4,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96425684766615, 28.660484181792977]),
            {
              "Class": 4,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96756937579954, 28.659215596039203]),
            {
              "Class": 4,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([76.96553089694822, 28.65995227201013]),
            {
              "Class": 4,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97009071414352, 28.658713304559555]),
            {
              "Class": 4,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([76.97416767184616, 28.65820492207687]),
            {
              "Class": 4,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98487775184043, 28.650113544208136]),
            {
              "Class": 4,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98398725844748, 28.651506990439106]),
            {
              "Class": 4,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98610083915122, 28.648258725567096]),
            {
              "Class": 4,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([76.98913709975608, 28.646639263178475]),
            {
              "Class": 4,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99235575057395, 28.64614965334757]),
            {
              "Class": 4,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99705498076804, 28.645443481296848]),
            {
              "Class": 4,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99779527045615, 28.647006469051785]),
            {
              "Class": 4,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99929730750449, 28.651450499735805]),
            {
              "Class": 4,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00163317775656, 28.65605121631518]),
            {
              "Class": 4,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0026846036904, 28.657990627947157]),
            {
              "Class": 4,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00817777880381, 28.66407728356954]),
            {
              "Class": 4,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00711025961589, 28.664449135350523]),
            {
              "Class": 4,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00838699110697, 28.665357577823695]),
            {
              "Class": 4,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0086444831724, 28.664642121197815]),
            {
              "Class": 4,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00598909624766, 28.664402065577743]),
            {
              "Class": 4,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00504495867442, 28.662773438421038]),
            {
              "Class": 4,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01499346208438, 28.664086609357458]),
            {
              "Class": 4,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01640698623522, 28.663036944773737]),
            {
              "Class": 4,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01561627962244, 28.678726838910812]),
            {
              "Class": 4,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03503968773897, 28.690194291113677]),
            {
              "Class": 4,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03554553098294, 28.691692092744027]),
            {
              "Class": 4,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01469659762695, 28.702602353188823]),
            {
              "Class": 4,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01298579354903, 28.70786510423701]),
            {
              "Class": 4,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01636740550305, 28.714086093771346]),
            {
              "Class": 4,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01871702060009, 28.717962731983086]),
            {
              "Class": 4,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02165672168042, 28.721481691752167]),
            {
              "Class": 4,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03138912633456, 28.721702897590248]),
            {
              "Class": 4,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Point([77.03527296498812, 28.72118541202839]),
            {
              "Class": 4,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0617884906477, 28.74330411223112]),
            {
              "Class": 4,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25255656815585, 28.693104791200234]),
            {
              "Class": 4,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25256193257388, 28.691269556527608]),
            {
              "Class": 4,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Point([77.23341743659473, 28.709671733368094]),
            {
              "Class": 4,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Point([77.25044602631284, 28.726755987785516]),
            {
              "Class": 4,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1692659474262, 28.804482529697086]),
            {
              "Class": 4,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07899881023259, 28.81603409716496]),
            {
              "Class": 4,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07746458667607, 28.817133926914472]),
            {
              "Class": 4,
              "system:index": "47"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07874131816716, 28.814718046303984]),
            {
              "Class": 4,
              "system:index": "48"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08423448222966, 28.815545280214913]),
            {
              "Class": 4,
              "system:index": "49"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09727525943389, 28.848711887137544]),
            {
              "Class": 4,
              "system:index": "50"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12919423805057, 28.834523772208154]),
            {
              "Class": 4,
              "system:index": "51"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13057825790226, 28.83287899815467]),
            {
              "Class": 4,
              "system:index": "52"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13128367887317, 28.832815556349114]),
            {
              "Class": 4,
              "system:index": "53"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20564745040612, 28.724329968465938]),
            {
              "Class": 4,
              "system:index": "54"
            }),
        ee.Feature(
            ee.Geometry.Point([77.20208815904336, 28.72334206408444]),
            {
              "Class": 4,
              "system:index": "55"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19585103533281, 28.726769604215384]),
            {
              "Class": 4,
              "system:index": "56"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19310470331045, 28.729384873324364]),
            {
              "Class": 4,
              "system:index": "57"
            }),
        ee.Feature(
            ee.Geometry.Point([77.19497688520285, 28.73043857675445]),
            {
              "Class": 4,
              "system:index": "58"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09021671847748, 28.667964524342565]),
            {
              "Class": 4,
              "system:index": "59"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09169976557001, 28.668225752434385]),
            {
              "Class": 4,
              "system:index": "60"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08832184528448, 28.669375460590445]),
            {
              "Class": 4,
              "system:index": "61"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08827356552221, 28.669135415812704]),
            {
              "Class": 4,
              "system:index": "62"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08822796796896, 28.668690625506393]),
            {
              "Class": 4,
              "system:index": "63"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08852837537863, 28.669264851790572]),
            {
              "Class": 4,
              "system:index": "64"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09143857215979, 28.66829054795665]),
            {
              "Class": 4,
              "system:index": "65"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09179798816778, 28.668102275640024]),
            {
              "Class": 4,
              "system:index": "66"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12126427953933, 28.72906548436956]),
            {
              "Class": 4,
              "system:index": "67"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99955857100053, 28.627757823091862]),
            {
              "Class": 4,
              "system:index": "68"
            }),
        ee.Feature(
            ee.Geometry.Point([76.9980698391341, 28.62009831776187]),
            {
              "Class": 4,
              "system:index": "69"
            }),
        ee.Feature(
            ee.Geometry.Point([76.99945214460487, 28.626767169977352]),
            {
              "Class": 4,
              "system:index": "70"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02392088511448, 28.602061463327768]),
            {
              "Class": 4,
              "system:index": "71"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02370094397526, 28.600902852214467]),
            {
              "Class": 4,
              "system:index": "72"
            }),
        ee.Feature(
            ee.Geometry.Point([77.027337590414, 28.600195999112717]),
            {
              "Class": 4,
              "system:index": "73"
            }),
        ee.Feature(
            ee.Geometry.Point([77.02834073658558, 28.599640234290707]),
            {
              "Class": 4,
              "system:index": "74"
            }),
        ee.Feature(
            ee.Geometry.Point([77.00902883167835, 28.59455576525382]),
            {
              "Class": 4,
              "system:index": "75"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0092541372356, 28.59455576525382]),
            {
              "Class": 4,
              "system:index": "76"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0090020095882, 28.59229959754453]),
            {
              "Class": 4,
              "system:index": "77"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01431034345246, 28.57145072650277]),
            {
              "Class": 4,
              "system:index": "78"
            }),
        ee.Feature(
            ee.Geometry.Point([77.015128417202, 28.571269347200403]),
            {
              "Class": 4,
              "system:index": "79"
            }),
        ee.Feature(
            ee.Geometry.Point([77.01493261594392, 28.572112640524995]),
            {
              "Class": 4,
              "system:index": "80"
            }),
        ee.Feature(
            ee.Geometry.Point([76.95799096413775, 28.58053348454725]),
            {
              "Class": 4,
              "system:index": "81"
            }),
        ee.Feature(
            ee.Geometry.Point([76.95738478490038, 28.57978682962453]),
            {
              "Class": 4,
              "system:index": "82"
            }),
        ee.Feature(
            ee.Geometry.Point([76.94769425971617, 28.585491998750133]),
            {
              "Class": 4,
              "system:index": "83"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05721239565672, 28.756294977064943]),
            {
              "Class": 4,
              "system:index": "84"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07950139946647, 28.78293031236449]),
            {
              "Class": 4,
              "system:index": "85"
            }),
        ee.Feature(
            ee.Geometry.Point([77.07807446427054, 28.785723052145524]),
            {
              "Class": 4,
              "system:index": "86"
            }),
        ee.Feature(
            ee.Geometry.Point([77.08608632972293, 28.797034158540743]),
            {
              "Class": 4,
              "system:index": "87"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09575633842115, 28.80127584041368]),
            {
              "Class": 4,
              "system:index": "88"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10750797651731, 28.804378326796545]),
            {
              "Class": 4,
              "system:index": "89"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11670829008291, 28.80633935597528]),
            {
              "Class": 4,
              "system:index": "90"
            }),
        ee.Feature(
            ee.Geometry.Point([77.12371422002981, 28.807928143074975]),
            {
              "Class": 4,
              "system:index": "91"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09618659934901, 28.848753540031893]),
            {
              "Class": 4,
              "system:index": "92"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09656076750659, 28.848745317350975]),
            {
              "Class": 4,
              "system:index": "93"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09691079578303, 28.84876411204925]),
            {
              "Class": 4,
              "system:index": "94"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09739493451022, 28.848584387607954]),
            {
              "Class": 4,
              "system:index": "95"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09737041543865, 28.84824490726036]),
            {
              "Class": 4,
              "system:index": "96"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09738248537921, 28.84846809515689]),
            {
              "Class": 4,
              "system:index": "97"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13386939082716, 28.816264067219546]),
            {
              "Class": 4,
              "system:index": "98"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13372455154035, 28.816144213540465]),
            {
              "Class": 4,
              "system:index": "99"
            }),
        ee.Feature(
            ee.Geometry.Point([77.13368700061415, 28.816017309494626]),
            {
              "Class": 4,
              "system:index": "100"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11186005475854, 28.827615848631908]),
            {
              "Class": 4,
              "system:index": "101"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11211218240594, 28.82765109584491]),
            {
              "Class": 4,
              "system:index": "102"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11204244497155, 28.82718583167251]),
            {
              "Class": 4,
              "system:index": "103"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11232944133614, 28.827239877617462]),
            {
              "Class": 4,
              "system:index": "104"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1151042580195, 28.836221660223437]),
            {
              "Class": 4,
              "system:index": "105"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1161127686091, 28.83617936704913]),
            {
              "Class": 4,
              "system:index": "106"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11209994379749, 28.835576196984043]),
            {
              "Class": 4,
              "system:index": "107"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11398017231693, 28.835954487513966]),
            {
              "Class": 4,
              "system:index": "108"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11558263504456, 28.837436887523126]),
            {
              "Class": 4,
              "system:index": "109"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11443811979943, 28.83781759504235]),
            {
              "Class": 4,
              "system:index": "110"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11809665289574, 28.837897480877317]),
            {
              "Class": 4,
              "system:index": "111"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1180537375515, 28.838094845618407]),
            {
              "Class": 4,
              "system:index": "112"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11873816231508, 28.838000862455054]),
            {
              "Class": 4,
              "system:index": "113"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11987810114641, 28.837829342963104]),
            {
              "Class": 4,
              "system:index": "114"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11647984862245, 28.84297603261015]),
            {
              "Class": 4,
              "system:index": "115"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11649325966752, 28.84322742536082]),
            {
              "Class": 4,
              "system:index": "116"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11565413987623, 28.844074496043753]),
            {
              "Class": 4,
              "system:index": "117"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11503186738477, 28.844363477130234]),
            {
              "Class": 4,
              "system:index": "118"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1168021253346, 28.844307090639823]),
            {
              "Class": 4,
              "system:index": "119"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11692282474027, 28.843966421610023]),
            {
              "Class": 4,
              "system:index": "120"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11492652143995, 28.84318946540548]),
            {
              "Class": 4,
              "system:index": "121"
            }),
        ee.Feature(
            ee.Geometry.Point([77.1154575988249, 28.842578603237744]),
            {
              "Class": 4,
              "system:index": "122"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11649829592268, 28.842745415954926]),
            {
              "Class": 4,
              "system:index": "123"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11908662762205, 28.85274664174363]),
            {
              "Class": 4,
              "system:index": "124"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11681270334535, 28.854220345199828]),
            {
              "Class": 4,
              "system:index": "125"
            }),
        ee.Feature(
            ee.Geometry.Point([77.11837911341004, 28.852636963030996]),
            {
              "Class": 4,
              "system:index": "126"
            }),
        ee.Feature(
            ee.Geometry.Point([77.10148843850642, 28.739902136418916]),
            {
              "Class": 4,
              "system:index": "127"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09394606675654, 28.739556422460883]),
            {
              "Class": 4,
              "system:index": "128"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0927358219989, 28.739565481690658]),
            {
              "Class": 4,
              "system:index": "129"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09265535572845, 28.73996058322932]),
            {
              "Class": 4,
              "system:index": "130"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09266608456451, 28.73970423775839]),
            {
              "Class": 4,
              "system:index": "131"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09389185408432, 28.739727755717688]),
            {
              "Class": 4,
              "system:index": "132"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09253197411377, 28.739344412320715]),
            {
              "Class": 4,
              "system:index": "133"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09392672280151, 28.74002172976252]),
            {
              "Class": 4,
              "system:index": "134"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09386234978516, 28.74014637450781]),
            {
              "Class": 4,
              "system:index": "135"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09407960871536, 28.739483168682053]),
            {
              "Class": 4,
              "system:index": "136"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09354316691238, 28.739462002469335]),
            {
              "Class": 4,
              "system:index": "137"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09406083325226, 28.739431429043425]),
            {
              "Class": 4,
              "system:index": "138"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09485476712067, 28.73948081688084]),
            {
              "Class": 4,
              "system:index": "139"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09526246289093, 28.737416103075144]),
            {
              "Class": 4,
              "system:index": "140"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09537511566955, 28.737719490987857]),
            {
              "Class": 4,
              "system:index": "141"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09556823471863, 28.737957026645777]),
            {
              "Class": 4,
              "system:index": "142"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0960322568782, 28.73830744955122]),
            {
              "Class": 4,
              "system:index": "143"
            }),
        ee.Feature(
            ee.Geometry.Point([77.09634607533295, 28.7388765902892]),
            {
              "Class": 4,
              "system:index": "144"
            }),
        ee.Feature(
            ee.Geometry.Point([77.0962384737672, 28.737035054579515]),
            {
              "Class": 4,
              "system:index": "145"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05248035774768, 28.715163445517877]),
            {
              "Class": 4,
              "system:index": "146"
            }),
        ee.Feature(
            ee.Geometry.Point([77.05258764610828, 28.71647134303782]),
            {
              "Class": 4,
              "system:index": "147"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06072884908768, 28.712421347060218]),
            {
              "Class": 4,
              "system:index": "148"
            }),
        ee.Feature(
            ee.Geometry.Point([77.06207263580414, 28.711769727446843]),
            {
              "Class": 4,
              "system:index": "149"
            })]);
